In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = '/content/drive/MyDrive/Emotion_recognition/FER-2013/Train'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:
test_path ='/content/drive/MyDrive/Emotion_recognition/FER-2013/Test'

In [8]:
if tf.test.gpu_device_name():
    print('GPU device found:', tf.test.gpu_device_name())
else:
    print('No GPU device found')

GPU device found: /device:GPU:0


In [1]:
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.metrics import categorical_accuracy

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [2]:
from tensorflow.keras.models import load_model

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

In [3]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [1]:
import keras
print(keras.__version__)

2.15.0


## FUNCTIONS

In [10]:
def read_images(training_dir, categories, img_size=48):
    images = []
    img_size = img_size

    for cat in categories:
        path = os.path.join(training_dir, cat)
        class_num = categories.index(cat)
        print(cat, class_num)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_img = cv2.resize(img_array, (img_size, img_size))
            gray_image = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
            images.append([gray_image, class_num])
    return images

In [11]:
def split_images_into_X_y(images, flatten = False, img_size =48):

    img_size = img_size
    X = []
    y = []

    for pict, label in images:
        X.append(pict)
        y.append(label)

    X = np.array(X)
    y = np.array(y)

    X = X / 255.0

    if flatten:
        X_flatten = X.reshape((-1, img_size*img_size))
        return X_flatten, y
    else:
        return X, y

# Train, test, validation split

In [12]:
categories = ['0', '1', '2', '3', '4', '5', '6']

In [13]:
images = read_images(file_path, categories, img_size=48)

0 0
1 1
2 2
3 3
4 4
5 5
6 6


In [14]:
X_train, y_train = split_images_into_X_y(images, flatten = False, img_size =48)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [15]:
X_train.shape

(21611, 48, 48)

In [16]:
y_train.shape

(21611,)

In [22]:
test_images = read_images(test_path, categories, img_size=48)

0 0
1 1
2 2
3 3
4 4
5 5
6 6


In [23]:
X_test, y_test = split_images_into_X_y(test_images, flatten = False, img_size =48)

In [24]:
X_test.shape, y_test.shape

((7178, 48, 48), (7178,))

# Model deep 48x48 pixels

In [17]:
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

# model.compile(, loss='binary_crossentropy', metrics=[categorical_accuracy])

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [19]:
history = model.fit(
    X_train,
    to_categorical(y_train),
    epochs=124,
    batch_size=128,
    #validation_data=(X_val, to_categorical(y_val))
)

Epoch 1/124
169/169 [==============================] - 25s 76ms/step - loss: 1.9129 - categorical_accuracy: 0.2516
Epoch 2/124
169/169 [==============================] - 11s 63ms/step - loss: 1.5843 - categorical_accuracy: 0.3793
Epoch 3/124
169/169 [==============================] - 11s 63ms/step - loss: 1.4061 - categorical_accuracy: 0.4546
Epoch 4/124
169/169 [==============================] - 11s 63ms/step - loss: 1.3080 - categorical_accuracy: 0.4953
Epoch 5/124
169/169 [==============================] - 11s 62ms/step - loss: 1.2289 - categorical_accuracy: 0.5280
Epoch 6/124
169/169 [==============================] - 11s 62ms/step - loss: 1.1656 - categorical_accuracy: 0.5549
Epoch 7/124
169/169 [==============================] - 10s 62ms/step - loss: 1.1238 - categorical_accuracy: 0.5758
Epoch 8/124
169/169 [==============================] - 10s 61ms/step - loss: 1.0618 - categorical_accuracy: 0.5977
Epoch 9/124
169/169 [==============================] - 10s 62ms/step - loss: 1.0

In [20]:
model.save('/content/drive/MyDrive/Emotion_recognition/FER-2013/CNN_deep_github.keras')

In [25]:
test_loss, test_accuracy = model.evaluate(X_test, to_categorical(y_test))

225/225 [==============================] - 3s 8ms/step - loss: 2.4800 - categorical_accuracy: 0.6101


In [4]:
model.save('/content/drive/MyDrive/Emotion_recognition/FER-2013/CNN_48x48.h5')

NameError: name 'model' is not defined

In [26]:
y_pred = model.predict(X_test)

y_pred_labels = np.argmax(y_pred, axis=1)

225/225 [==============================] - 2s 6ms/step


In [28]:
y_pred_labels

array([0, 6, 0, ..., 4, 6, 3])

In [30]:
cf_model = confusion_matrix(y_test, y_pred_labels)
cf_model

array([[ 512,    7,   62,   54,  159,   18,  146],
       [  15,   61,    5,    4,   18,    0,    8],
       [  97,    0,  422,   56,  234,   57,  158],
       [  56,    0,   46, 1373,  111,   17,  171],
       [ 145,    2,  107,   71,  643,    6,  273],
       [  31,    0,   79,   39,   35,  595,   52],
       [ 104,    1,   49,   91,  210,    5,  773]])

In [6]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Emotion_recognition/FER-2013/CNN_deep_github.keras')

In [7]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

# Model 2 - 224x224 pixels

In [35]:
def split_images_into_X_y_resize(images, img_size=224):
    X = []
    y = []

    for pict, label in images:
        new_img = cv2.resize(pict, (img_size, img_size), interpolation=cv2.INTER_LANCZOS4)
        X.append(new_img)
        y.append(label)

    X = np.array(X, dtype=np.float32) / 255.0
    y = np.array(y)

    return X, y

In [ ]:
X_train_224, y_train_224 = split_images_into_X_y_resize(images, img_size =224)

In [34]:
X_train_224.shape,   y_train_224.shape

((21611, 224, 224), (21611,))

In [8]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [9]:
import keras
print(keras.__version__)

2.15.0
